# Orbital-Optimization for reducing the one-norm

Author : Dr. Saad Yalouz - MSc Emiel Koridon (Cool titles)

A starting point to minimize the One-norm using Orbital-Optimization and a gradient free optimizer

## Theory 
The idea here is to employ a unitary transformation to modify the MOs of the system in order to minimize the One-norm. To build up this unitary, we start by the definition of the Orbital rotation operator which reads
$$ U(\vec{\kappa}) =\sum_{t>u}^{active} \kappa_{tu} (\hat{E}_{tu} - \hat{E}_{ut})$$
with $\hat{E}_{tu}=\frac{1} {\sqrt{2}}\sum_{\sigma}a^\dagger_{t,\sigma} a_{u,\sigma}$ and $\kappa_{tu}$ a rotation parameter affecting MOs $t$ and $u$ (note the absence of self rotation here with $t\neq u$). This operator takes the form of single excitations (with $t,u$ indices running only in the active space). Starting from this definition, a little bit of aglebra can show that implementing this rotation operator on creation/anihilation operators is strictly equivalent to redefining the MO coeff matrix of the Hamiltonian as follows
$$ \mathbf{C}_{new}(\vec{\kappa}) = \mathbf{C}_{ref} \mathbf{U}_{OO}(\vec{\kappa}) $$ 
Here, $\mathbf{U}_{OO}(\vec{\kappa})$ is a matrix representation of the effect of the rotations operator on the MOs. This matrix is an exponentiated version of the skew-matrix noted $\mathbf{K}$ encoding the parameter $\vec{\kappa}$. In other words
$$\mathbf{U}_{OO}(\vec{\kappa}) = e^{-\mathbf{K}} \text{,  with  } \mathbf{K} = skew(\vec{\kappa}) = \pmatrix{ 0 & \kappa_{01} & \kappa_{02} & \ldots & \kappa_{0N} \\
-\kappa_{01} & 0 & \kappa_{12} & \ldots & \kappa_{1N} \\
-\kappa_{02} & -\kappa_{12} & 0 &\ldots & \vdots \\
\vdots & \vdots & \vdots &\ddots & \vdots \\
-\kappa_{0N} & -\kappa_{1N} & \ldots & \ldots & 0 }$$

## Numerical implementation

Starting from this, we can encode this exponential numerically and use the kappa parameters (total number of parameters = $N(N-1)/2$) as degrees of freedom to reduce the one-norm. For this, first I use scimpy symbols to encode the matrix.

In [ ]:
import psi4
import sys
import os
import openfermion
from openfermionpyscf import run_pyscf, compute_integrals
import numpy as np
import copy
from scipy.optimize import minimize
import math as m
import scipy 
import sympy
from sympy.matrices import zeros
from random import random
from openfermion import ops, InteractionOperator, get_sparse_operator, jordan_wigner
from openfermion.functionals import JW1norm_spat
from pyscf import gto, scf, lo, tools, fci, ao2mo, mcscf, df
import module as md
import time

# Set molecule parameters.
basis = ['sto-3g', 'cc-pvdz','def2svp'][0]
multiplicity = 1

# Set calculation parameters.
consider_cas = 1 # Do we consider an active space or the full space?
# Set size of active space
n_orbitals = 8 
n_electrons = 8
# Choose whether you want to start from localized orbitals or canonical orbitals
localize = 0
# Import .xyz geometry. This is kind of an involved code because I have many different geometries available...
# If you want to consider a Hydrogen chain or ring, set the following to 1
H_chain = 0
H_ring = 0
run_fci = 0
run_casci = 0
if H_chain:
    N_atoms = 14
    r = 1.8
    if consider_cas:
        description = 'H' + str(N_atoms) + str(basis) + 'ne' + str(n_electrons) +\
            'no' + str(n_orbitals)
    else:
        description = 'H' + str(N_atoms) + str(basis)
    geometry = [('H',( 0., 0., z*r)) for z in range(N_atoms)]
    
elif H_ring:
    theta = 88 * np.pi/180
    r = 1.3
    # Introduction of the molecular structure (.txt file)
    geometry = [
                ('H', (r*np.cos(theta/2.),   r*np.sin(theta/2.),  0.)),
                ('H', (r*np.cos(theta/2.),   -r*np.sin(theta/2.), 0.)),
                ('H', (-r*np.cos(theta/2.),  r*np.sin(theta/2.),  0.)),
                ('H', (-r*np.cos(theta/2.),  -r*np.sin(theta/2.), 0.))
                ]
    if consider_cas:
        description = 'H4' + str(basis) + 'ne' + str(n_electrons) + 'no' + str(n_orbitals)
    else:
        description = 'H4' + str(basis)

else:
    fname = ['xyz_files/H2nosym.txt','xyz_files/H2COnosym.txt','xyz_files/H10.txt',\
             'xyz_files/C2.txt', 'xyz_files/LiH.txt', 'xyz_files/HLiO.txt', \
             'xyz_files/H2Onosym.txt', 'xyz_files/H14.txt', \
             'xyz_files/hnch2_s0min_dzp.txt', 'xyz_files/hnc3h6_s0min_dzp.txt',\
             'xyz_files/hnc5h10_s0min_dzp.txt', 'xyz_files/hnc7h14_s0min_dzp.txt',\
             'xyz_files/benzene.txt','xyz_files/PCy3.txt','xyz_files/PCy3Cl2Ru.txt'][8]
    geometry = md.xyz_to_geom(fname)
    if consider_cas:
        description = fname.replace('xyz_files/','').replace('.txt','') + str(basis)\
            + 'ne' + str(n_electrons) + 'no' + str(n_orbitals)
    else:
        description = fname.replace('xyz_files/','').replace('.txt','') + str(basis)

# Determine number of AOs and electrons
nao = md.count_ao(geometry,basis,spin=multiplicity-1)
nelec = md.count_elec(geometry,basis,spin=multiplicity-1)
nmo = nao
print('nao:',nao,'\nnelec:', nelec)
threshold = 1e-10

# Set active indices
if consider_cas:
    ncore = (nelec - n_electrons) // 2
    ntot = ncore + n_orbitals # Number of core orbitals + number of active orbitals
    active_indices = list(range(ncore,ntot))
else:
    ncore = 0
    ntot = nmo
    active_indices = list(range(nmo))

# Build molecule and run RHF
mol = gto.Mole()
mol.atom = geometry
mol.spin = multiplicity - 1
mol.symmetry = 0
mol.build()

if multiplicity == 1:
    myhf = scf.RHF(mol)
else:
    myhf = scf.ROHF(mol)

myhf.run()

# Extract MO_coeff and integrals. Then determine qubit 1-norm for CMOs and Pipek-Mezey MOs.

C_nonloc = np.copy(myhf.mo_coeff)

constant = float(mol.energy_nuc())


print('---------NON-LOCALIZED_ORBITALS---------')

C_copy = np.copy(C_nonloc)
one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_copy[:,ncore:ntot], threshold)

t4 = time.time()
qub1norm = JW1norm_spat(np.copy(constant),
                        np.copy(one_body_integrals),
                        np.copy(two_body_integrals))
print('\ncalculating norm of qubit hamiltonian took', time.time()-t4)

print('\n')

print('---------LOCALIZED_ORBITALS---------')
localizemethod = 'Pipek-Mezey'

C = np.copy(C_nonloc)
if localizemethod == 'Pipek-Mezey':
    orb = lo.PipekMezey(mol).kernel(C[:,ncore:ntot])
elif localizemethod == 'Boys':
    orb = lo.Boys(mol).kernel(C[:,ncore:ntot])
elif localizemethod == 'ibo':
    orb = lo.ibo.ibo(mol, C[:,ncore:ntot])
elif localizemethod == 'ER':
    orb = lo.EdmistonRuedenberg(mol).kernel(C[:,ncore:ntot])
else:
    raise ValueError('Localization method not recognized')

# C_locPM = np.hstack((C_nonloc[:,:ncore],orb,C_nonloc[:,ntot:nmo]))
one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, orb, threshold)

t4 = time.time()
qub1norm_locPM = JW1norm_spat(np.copy(constant),
                              np.copy(one_body_integrals),
                              np.copy(two_body_integrals))
print('calculating norm of qubit hamiltonian took', time.time()-t4)

if consider_cas:
    print("Considering 1-norm of active space Hamiltonian")
else:
    print("Considering 1-norm of full space Hamiltonian")
    
print("starting 1-norm nonloc is:",qub1norm)
print("1norm locPM is:",qub1norm_locPM)


OPT_OO_MAX_ITER  = 50000 # Maximum number of steps for the OO

# Building the K matrix : generator for the MO rotations 
Rot_param_values = []
Rot_param_names  = []
bounds = (-1.,1.)

K_mat_sym = zeros( nmo )
bounds = []
for q in range(nmo-1):
    for p in range(q+1,nmo):
        if ( p in active_indices and q in active_indices ):
            K_mat_sym[ p, q ] = - sympy.Symbol( "K_{}{}".format( p, q ) )
            K_mat_sym[ q, p ] = - K_mat_sym[ p, q ] 
            Rot_param_names  += [sympy.Symbol( "K_{}{}".format( p, q ))]
            Rot_param_values += [0 * (random()-0.5)] # <== Set starting Rot_param_values here.
            bounds += [ [-1., 1.] ] 

# Building the bounds for the rotational parameter amplitudes in the form of constraints
# (simply beacause cobyla doesn't have intrinsic bound options)
cons = []
for factor in range(len(bounds)):
    lower, upper = bounds[factor]
    l = {'type': 'ineq',
         'fun': lambda x, lb=lower, i=factor: x[i] - lb}
    u = {'type': 'ineq',
         'fun': lambda x, ub=upper, i=factor: ub - x[i]}
    cons.append(l)
    cons.append(u)


In [ ]:
K      = np.array( K_mat_sym.subs(list(zip(Rot_param_names,Rot_param_values))) ).astype(np.float64)  
U_OO   = scipy.linalg.expm( - K )

print("check if U_OO is unitary:",np.allclose(np.eye(U_OO.shape[0]), U_OO @ U_OO.T))

Then I define a cost function to encode the evaluation of the One-Norm using the new MO coeff matrix. In the latter, we compute the 1-norm from the integrals. To proceed, we use $\mathbf{C}_{new}(\vec{\kappa})$ to build up the new electronic integrals.

In [ ]:
def K_matr(Rot_param_values, active_indices):
    K = zeros( nmo )
    n = 0
    for q in range(nmo-1):
        for p in range(q+1,nmo):
            if ( p in active_indices and q in active_indices ):
                K[ p, q ] = - Rot_param_values[n]
                K[ q, p ] = - K[ p, q ]
                n += 1
    return np.array(K)

def Cost_function_OO_OneNorm(Rot_param_values):
#                              Rot_param_names,
#                              K_matr):
    """
    Cost function to minimize the One-Norm using MO rotations.
    """
    t6 = time.time()
    t5 = time.time()
    K = K_matr(Rot_param_values, active_indices)
    print("\nInitializing K-matrix took:",time.time()-t5)
    t1 = time.time()
    U_OO   = scipy.linalg.expm( - K )
    print("exponentiating matrix took:",time.time()-t1)
    t3 = time.time()
    if localize:
        C_MO   = C_locPM @ U_OO
    else:
        C_MO   = C_nonloc @ U_OO
    print("Matrix multiplication took:",time.time()-t3)
    t2 = time.time()
    one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_MO[:,ncore:ntot], threshold)
    print("extracting integrals took:",time.time()-t2)
    t4 = time.time()
    OneNorm = JW1norm_spat(np.copy(constant),
                           np.copy(one_body_integrals),
                           np.copy(two_body_integrals))
    print('calculating norm of qubit hamiltonian took', time.time()-t4)
    print('1-norm is:',OneNorm)
    print("total time for 1 cost function evaluation is", time.time()-t6)
    return OneNorm

print("starting 1-norm nonloc is:",qub1norm)
print("1norm locPM is:",qub1norm_locPM)
t7 = time.time()
f_min_OO = minimize( Cost_function_OO_OneNorm,
                      x0      = Rot_param_values,
#                       args    = (Rot_param_names,
#                                  K_mat_sym),
                      constraints=cons,
                      method  = 'cobyla',
                      options = {'maxiter': OPT_OO_MAX_ITER,
                                'tol'    : 1e-5,
                                'disp': True}  )
print("total time for minimization with",OPT_OO_MAX_ITER,"max iterations was:", time.time()-t7)

In [ ]:
print("message:",f_min_OO.message,"number of function evaluations:",f_min_OO.nfev)

In [ ]:
K = K_matr(f_min_OO.x, active_indices)
U_OO   = scipy.linalg.expm( - K )
if localize:
    C_OO = C_locPM @ U_OO
else:
    C_OO = C_nonloc @ U_OO

one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_OO[:,ncore:ntot], threshold)

OneNormorbOO = JW1norm_spat(np.copy(constant),
                       np.copy(one_body_integrals),
                       np.copy(two_body_integrals))

one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_nonloc[:,ncore:ntot], threshold)

OneNormorbnonloc = JW1norm_spat(np.copy(constant),
                       np.copy(one_body_integrals),
                       np.copy(two_body_integrals))

one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_locPM[:,ncore:ntot], threshold)

OneNormorblocPM = JW1norm_spat(np.copy(constant),
                       np.copy(one_body_integrals),
                       np.copy(two_body_integrals))
if consider_cas:
    print("Considering CAS(" + str(n_electrons) + "," + str(n_orbitals) + "), \n1norm of CMO orb is",
         OneNormorbnonloc, "\n1norm of PM orb is", OneNormorblocPM, "\n1norm of optimizer is", OneNormorbOO)
else:
    print("Considering full space, \n1norm of CMO orb is",
         OneNormorbnonloc, "\n1norm of PM orb is", OneNormorblocPM, "\n1norm of optimizer is", OneNormorbOO)

## Visualize
If you want to save the orbitals to gaussian CUBE files

In [ ]:
Cost_function_OO_OneNorm(f_min_OO.x)
K = K_matr(f_min_OO.x, active_indices)
U_OO   = scipy.linalg.expm( - K )
if localize:
    C_OO = C_locPM @ U_OO
else:
    C_OO = C_nonloc @ U_OO
t13 = time.time()
for i in range(nmo):    
    tools.cubegen.orbital(mol, os.getcwd() + '/CUBE_FILES/pyscfcube'\
                          + description + 'onenorm_orb' + localizemethod + str(localize)\
                          + str(consider_cas) + str(i) , C_OO[:,i])
print('Cube files of molecule', description,'created in', os.getcwd() + '/CUBE_FILES/')
print('extracting cube files took', time.time()-t13)